In [187]:
import pandas as pd
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import xgboost as xgb
from xgboost import XGBClassifier, XGBRanker, XGBRegressor

import catboost
from catboost import CatBoostRanker, Pool

import lightgbm as lgb

from sklearn.model_selection import StratifiedGroupKFold


In [188]:
data = pd.read_csv('intern_task.csv')
data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000


In [189]:
data.describe()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
count,125211.000000,125211.000000,125211.000000,125211.000000,125211.000000,125211.000000,125211.000000,125211.000000,125211.000000,125211.000000,...,1.252100e+05,1.252100e+05,125210.000000,125210.000000,125210.000000,125210.000000,125210.000000,125210.000000,125210.000000,125210.000000
mean,0.645095,8287.490716,1.940940,0.212306,1.197499,0.543075,1.984243,0.805074,0.098596,0.533730,...,5.843738e+02,3.205094e+03,0.068916,0.738888,11.010490,0.269927,0.026304,0.000283,22.408370,9.518075
std,0.815760,4582.962301,1.187071,0.587094,1.031456,0.791812,1.155348,0.335066,0.265577,0.416526,...,2.251566e+04,1.130422e+06,0.100907,0.348763,13.774049,0.383365,0.032479,0.002174,35.685266,14.598181
min,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4300.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.666667,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.614251,0.000000,0.000000,0.006553,0.000000,4.000000,2.000000
50%,0.000000,8590.000000,2.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.500000,...,0.000000e+00,0.000000e+00,0.000000,0.917520,9.304890,0.000000,0.017371,0.000010,12.000000,5.500000
75%,1.000000,12235.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1.000000,...,1.000000e+00,6.000000e-01,0.133333,0.998442,16.210163,0.500000,0.033853,0.000117,28.000000,12.000000
max,4.000000,15850.000000,8.000000,6.000000,6.000000,6.000000,8.000000,1.000000,1.000000,1.000000,...,1.731553e+06,4.000000e+08,0.818182,1.000000,1301.405861,1.000000,0.711261,0.250000,3104.000000,1552.000000


In [190]:
data.query_id.nunique()

1057

## Preprocessing

Убираем слишком коррелированные данные

In [191]:
corr_matrix = data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

data.drop(columns=to_drop, inplace=True)

In [192]:
data.shape # половина features ушло

(125211, 82)

In [193]:
data.isnull().sum()

rank           0
query_id       0
feature_0      0
feature_1      0
feature_2      0
              ..
feature_132    1
feature_133    1
feature_134    1
feature_135    1
feature_141    1
Length: 82, dtype: int64

Мы видим, что есть пару стобцов с пропущенными значениями, их можно заменить means

In [194]:
data.fillna(data.mean(), inplace=True)

## Model

### CatBoostRanker

In [195]:
X = data.copy()
y = X.pop('rank')

sgkf = StratifiedGroupKFold(n_splits=5)

groups = X['query_id'].tolist()
g = sgkf.split(X, y, groups)

train_idx, val_idx = next(g)
X_train, X_test = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[val_idx]

queries_train = X_train['query_id']
queries_test = X_test['query_id']

X_train.drop(columns=['query_id'], inplace=True)
X_test.drop(columns=['query_id'], inplace=True)

train_pool = Pool(data=X_train,
                  label=y_train,
                  group_id=queries_train.tolist())
test_pool = Pool(data=X_test,
                label=y_test,
                  group_id=queries_test.tolist())

<ipython-input-195-67562efad66c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=['query_id'], inplace=True)
<ipython-input-195-67562efad66c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns=['query_id'], inplace=True)


In [196]:
rnkr = CatBoostRanker(custom_metric=['NDCG'],
                      random_state=42,
                      iterations=500,
                      loss_function='RMSE',
                      verbose=False)

rnkr.fit(train_pool, eval_set=test_pool)
rnkr.score(test_pool)

/usr/local/lib/python3.10/dist-packages/catboost/core.py:6346: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


0.7934888906728231

## LGBoosting

In [197]:
X = data.drop(columns=['query_id', 'rank'])
y = data['rank']
query_id = data['query_id']

X_train, X_test, y_train, y_test, queries_train, queries_test = train_test_split(X, y, query_id, test_size=0.2, random_state=42)

In [198]:
train_group = queries_train.value_counts().sort_index().values
test_group = queries_test.value_counts().sort_index().values

train_data = lgb.Dataset(X_train, label=y_train, group=train_group)
test_data = lgb.Dataset(X_test, label=y_test, group=test_group)

params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [5],
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 35,
    'min_data_in_leaf': 45,
    'num_iterations': 1000,
    'verbose': -1
}

model = lgb.train(
    params,
    train_data)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

ndcg_5 = ndcg_score([y_test], [y_pred], k=5)
ndcg_5

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


0.9576049743407978

## XGBoosting

In [199]:
xgb_rank = XGBRanker(n_estimators=500, learning_rate=0.1,
                        max_depth=7,
                        objective='rank:pairwise',
                        verbosity=0, use_label_encoder=False,
                        tree_method='hist', reg_alpha=0.005, verbose=True, device='gpu')

sorted_idx = queries_train.argsort()
X_train_sorted = X_train.iloc[sorted_idx]
y_train_sorted = y_train.iloc[sorted_idx]
query_id_train_sorted = queries_train.iloc[sorted_idx]
group_train = query_id_train_sorted.groupby(query_id_train_sorted).size().values

xgb_rank.fit(X_train_sorted, y_train_sorted, group=group_train)
y_pred_xgb_rank = xgb_rank.predict(X_test)

ndcg_5 = ndcg_score([y_test], [y_pred_xgb_rank], k=5)
ndcg_5

0.9465034338163679

LGBoosting оказался лучшим со скором 0.95